# Import Necessary libraries

In [1]:
import os
import re
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

# Load dataset

In [ ]:
data = pd.read_excel("C:\\Users\\akhil\\Downloads\\US_addresses.xlsx")

# Setting OpenApi key

In [ ]:
import os
os.environ["OPENAI_API_KEY"] ="your api key"

# Initialize OpenAI LLM

In [ ]:
llm = ChatOpenAI(model="gpt-4", temperature=0)

C:\Users\akhil\AppData\Local\Temp\ipykernel_6400\814062019.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0)


#  Define system message for LLM

In [ ]:

system_message = SystemMessage(
    content="""
    You are an expert in US geography. Your task is to fill missing values for **State, Country, and Place**
    based on the given **Street Name, Location, and Pincode**.
    - Extract the correct **State, Country, and Place**.
    - If unsure, return 'Unknown'.
    """
)

# Function to get missing details

In [ ]:
def fill_missing_details(row):
    if pd.isna(row['State']) or pd.isna(row['Country']) or pd.isna(row['Place']):
        address = f"{row['Street Name']}, {row['City']}, {row['Pincode']}"
        human_message = HumanMessage(content=f"Find the State, Country, and Place for this address: {address}")
        
        try:
            response = llm([system_message, human_message]).content
            
            #  Extract values from response
            lines = response.split("\n")
            state = next((line.split(": ")[1] for line in lines if "State:" in line), "Unknown")
            country = next((line.split(": ")[1] for line in lines if "Country:" in line), "Unknown")
            place = next((line.split(": ")[1] for line in lines if "Place:" in line), "Unknown")
            
            print(f"Processed: {address} -> State: {state}, Country: {country}, Place: {place}")
            return pd.Series([state, country, place])
        except Exception as e:
            print(f"Error processing {address}: {e}")
            return pd.Series([row['State'], row['Country'], row['Place']])
    else:
        return pd.Series([row['State'], row['Country'], row['Place']])

In [15]:
print(data.columns)

Index(['Company Name', 'Street Name', 'Place', 'City', 'Pincode', 'State',
       'County'],
      dtype='object')


#  Apply function to dataset

In [ ]:
data[['State', 'Country', 'Place']] = data.apply(fill_missing_details, axis=1)

C:\Users\akhil\AppData\Local\Temp\ipykernel_6400\3164494572.py:8: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([system_message, human_message]).content


Processed: 810 Crescent Centre Drive, Suite 400, Franklin, TN 37067 -> State: Tennessee, Country: United States, Place: Franklin
Processed: 888 W. Big Beaver Road, Suite 1200, Troy, MI 48084 -> State: Michigan, Country: United States, Place: Troy
Processed: 2 Tower Center Boulevard, Suite 2200, East Brunswick, NJ 08816 -> State: New Jersey, Country: United States, Place: East Brunswick
Processed: 1114 Avenue of the Americas, Suite 3030, New York, NY 10036 -> State: New York, Country: United States, Place: New York City
Processed: 2858 Woodcock Blvd, Suites 100-300, Atlanta, GA 30341 -> State: Georgia, Country: United States, Place: Atlanta
Processed: 5445 Legacy Drive, Suite 300, Plano, TX 75024 -> State: Texas, Country: United States, Place: Plano
Processed: 425 National Avenue, Mountain View, CA 94043 -> State: California, Country: United States, Place: Mountain View
Processed: 500 Frank W. Burr Blvd., Teaneck, NJ 07666 -> State: New Jersey, Country: United States, Place: Teaneck
Pro

# Save updated dataset

In [ ]:
data.to_excel("C:\\Users\\akhil\\Downloads\\US_addresses.xlsx", index=False)

print("Updated dataset saved successfully with missing details filled.")

Updated dataset saved successfully with missing details filled.
